In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [2]:
df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [3]:
colnames(data_source)

11-element Vector{Symbol}:
 :Yeld_10Y
 :Price_10Y
 :Stock_Price
 :Dividend
 :E
 :CAPE
 :T_Bill
 :CPI
 :Dividend_Yeld
 :Total_Stock_2
 :Total_Stock

In [4]:
freq = 4
#:Price_10Y
returns = percentchange(data_source[:Total_Stock], :log)
#returns = merge(returns, log1p.(data_source[:T_Bill])./freq)
cpi = percentchange(data_source[:CPI], :log)
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
term = data_source[:Yeld_10Y] ./freq .- data_source[:T_Bill] ./freq
dy = log.(data_source[:Dividend_Yeld])
data = merge(excess_returns, real_t_bill, term, cpi)

291×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1952-03-01 to 2024-07-01
┌────────────┬────────────────────┬─────────────┬─────────────────┬─────────────
│            │ Total_Stock_T_Bill │ T_Bill_CPI  │ Yeld_10Y_T_Bill │ CPI        ⋯
├────────────┼────────────────────┼─────────────┼─────────────────┼─────────────
│ 1952-03-01 │        -0.00772801 │  0.00772445 │       0.0022875 │ -0.0037807 ⋯
│ 1952-06-01 │          0.0651996 │ -0.00708528 │       0.0017875 │   0.011299 ⋯
│ 1952-09-01 │         -0.0227685 │  0.00423886 │       0.0024375 │         0. ⋯
│ 1952-12-01 │            0.08432 │   0.0089235 │       0.0011425 │ -0.0037523 ⋯
│ 1953-03-01 │         -0.0487217 │  0.00497517 │       0.0018125 │         0. ⋯
│ 1953-06-01 │        -0.00778441 │ -0.00227055 │       0.0021925 │  0.0074906 ⋯
│ 1953-09-01 │        -0.00277599 │ -0.00299956 │        0.002115 │  0.0074349 ⋯
│ 1953-12-01 │          0.0705234 │  0.00767892 │       0.0021725 │ -0.0037105 ⋯
│     ⋮      │         ⋮         

In [5]:
CSV.write("usa_data_transformed.csv", data)

"usa_data_transformed.csv"

In [48]:
start = Date(1998,01,01)
end_d = Date(2019, 12, 31)
s_data = to(from(data, start),end_d)

returns_summarystats(to(from(returns, start),end_d), freq)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1454,0.7736,0.074,0.252,-1.0147,-0.9472,-0.0626,0.1509,1.8816,0.5092,0.1428


In [49]:
returns_summarystats(s_data, 1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.0731,0.1825,0.0138,0.0616,-0.2565,-0.9474,-0.0186,0.1606,1.7048,0.1889,0.0305
T_Bill_CPI,0.0084,0.0255,-0.0007,0.0045,-0.0208,0.3107,-0.0052,0.0571,0.5427,-0.0815,-0.001
Yeld_10Y_T_Bill,0.0028,0.0095,0.0042,0.0063,-0.0016,-0.0991,0.002,0.8849,-0.9889,1.4761,0.0046
CPI,0.0079,0.0236,0.0053,0.0098,-0.0254,-0.6506,0.0018,-0.077,2.3017,0.671,0.0049


In [50]:
returns_summarystats(s_data, freq)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1462,0.7299,0.0552,0.2465,-1.026,-0.9474,-0.0746,0.1606,1.7048,0.3778,0.1218
T_Bill_CPI,0.0168,0.1019,-0.0027,0.0182,-0.0832,0.3107,-0.0209,0.0571,0.5427,-0.163,-0.0038
Yeld_10Y_T_Bill,0.0057,0.038,0.0168,0.0253,-0.0065,-0.0991,0.0078,0.8849,-0.9889,2.9523,0.0183
CPI,0.0158,0.0946,0.0213,0.0394,-0.1016,-0.6506,0.0071,-0.077,2.3017,1.342,0.0195


In [51]:
vmodel = VARModel(s_data)
model_summary(vmodel)

,const,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI,std
Total_Stock_T_Bill,0.0887,0.1337,-7.1367,-9.7153,-7.8291,0.0684
T_Bill_CPI,-0.0036,-0.0355,0.5888,-0.0585,0.7452,0.0073
Yeld_10Y_T_Bill,0.001,-0.0006,-0.0616,0.8271,-0.0492,0.0013
CPI,0.0034,0.0407,0.3961,0.087,0.2363,0.0074


,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI
Total_Stock_T_Bill,1.0,-0.07,0.14,0.09
T_Bill_CPI,-0.07,1.0,-0.17,-0.99
Yeld_10Y_T_Bill,0.14,-0.17,1.0,0.1
CPI,0.09,-0.99,0.1,1.0


In [52]:
bmodel = NormalWishartBVAR(s_data)
sample_posterior!(bmodel,10000, 2000)
posterior_summary(bmodel)

"Total_Stock_T_Bill coefficients"

Quantiles
          parameters       2.5%      25.0%     50.0%     75.0%     97.5% 
              Symbol    Float64    Float64   Float64   Float64   Float64 

               const     0.0358     0.0705    0.0893    0.1077    0.1431
  Total_Stock_T_Bill    -0.0729     0.0627    0.1337    0.2050    0.3390
          T_Bill_CPI   -12.0788    -8.8185   -7.1423   -5.4660   -2.2690
     Yeld_10Y_T_Bill   -17.5341   -12.4283   -9.8392   -7.0912   -1.9639
                 CPI   -12.8267    -9.5099   -7.8550   -6.1710   -2.9525


"T_Bill_CPI coefficients"

Quantiles
          parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
              Symbol   Float64   Float64   Float64   Float64   Float64 

               const   -0.0095   -0.0056   -0.0037   -0.0017    0.0021
  Total_Stock_T_Bill   -0.0581   -0.0434   -0.0356   -0.0280   -0.0130
          T_Bill_CPI    0.0640    0.4110    0.5910    0.7693    1.1155
     Yeld_10Y_T_Bill   -0.8810   -0.3358   -0.0548    0.2360    0.7969
                 CPI    0.2155    0.5668    0.7459    0.9278    1.2786


"Yeld_10Y_T_Bill coefficients"

Quantiles
          parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
              Symbol   Float64   Float64   Float64   Float64   Float64 

               const   -0.0001    0.0006    0.0010    0.0013    0.0019
  Total_Stock_T_Bill   -0.0043   -0.0018   -0.0005    0.0008    0.0033
          T_Bill_CPI   -0.1499   -0.0927   -0.0613   -0.0302    0.0297
     Yeld_10Y_T_Bill    0.6852    0.7776    0.8263    0.8754    0.9732
                 CPI   -0.1383   -0.0796   -0.0488   -0.0181    0.0429


"CPI coefficients"

Quantiles
          parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
              Symbol   Float64   Float64   Float64   Float64   Float64 

               const   -0.0023    0.0015    0.0035    0.0055    0.0094
  Total_Stock_T_Bill    0.0181    0.0332    0.0408    0.0487    0.0635
          T_Bill_CPI   -0.1423    0.2093    0.3942    0.5748    0.9250
     Yeld_10Y_T_Bill   -0.7783   -0.2124    0.0833    0.3681    0.9155
                 CPI   -0.3033    0.0494    0.2327    0.4153    0.7698


"correlation matrix"

Quantiles
                             parameters      2.5%     25.0%     50.0%     75.0 ⋯
                                 Symbol   Float64   Float64   Float64   Float6 ⋯

  Total_Stock_T_Bill_Total_Stock_T_Bill    0.0605    0.0665    0.0699    0.073 ⋯
          Total_Stock_T_Bill_T_Bill_CPI   -0.2780   -0.1410   -0.0676    0.008 ⋯
     Total_Stock_T_Bill_Yeld_10Y_T_Bill   -0.0827    0.0679    0.1410    0.217 ⋯
                 Total_Stock_T_Bill_CPI   -0.1275    0.0172    0.0922    0.165 ⋯
          T_Bill_CPI_Total_Stock_T_Bill   -0.2780   -0.1410   -0.0676    0.008 ⋯
                  T_Bill_CPI_T_Bill_CPI    0.0065    0.0071    0.0075    0.007 ⋯
             T_Bill_CPI_Yeld_10Y_T_Bill   -0.3663   -0.2355   -0.1672   -0.091 ⋯
                         T_Bill_CPI_CPI   -0.9951   -0.9934   -0.9923   -0.991 ⋯
     Yeld_10Y_T_Bill_Total_Stock_T_Bill   -0.0827    0.0679    0.1410    0.217 ⋯
             Yeld_10Y_T_Bill_T_Bill_CPI   -0.3663   -0.2355   -0.1672   -0.091 ⋯
        Yeld_10Y_

In [53]:
assets_names = colnames(s_data)
scenarios = simulate(vmodel, freq * 15)

for n in 1:length(assets_names)
   print_percentiles(scenarios[n, :, 2:end], [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], freq, string(assets_names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.1173,-0.208,-0.2358,-0.2473,-0.2529,-0.2574,-0.2562,-0.2569,-0.2541,-0.2551,-0.2547,-0.2415,-0.245,-0.2516,-0.2631
0.05,-0.0515,-0.1363,-0.1723,-0.1809,-0.1864,-0.1904,-0.1886,-0.1809,-0.1842,-0.1834,-0.1854,-0.179,-0.1833,-0.1884,-0.1906
0.25,0.0934,0.0086,-0.0207,-0.0289,-0.0329,-0.0304,-0.0296,-0.0283,-0.0303,-0.0256,-0.0286,-0.0239,-0.0221,-0.0282,-0.0272
0.5,0.1897,0.1141,0.0893,0.0812,0.0754,0.0805,0.0804,0.0805,0.0811,0.0863,0.0803,0.0854,0.0849,0.0845,0.0859
0.75,0.2852,0.2196,0.1974,0.1903,0.1874,0.1931,0.1893,0.1878,0.1929,0.1951,0.1962,0.1946,0.1954,0.1968,0.197
0.95,0.4213,0.3732,0.3548,0.345,0.3451,0.3495,0.3454,0.3478,0.3544,0.3515,0.3501,0.3504,0.3517,0.348,0.3568
0.98,0.4815,0.4416,0.4205,0.4116,0.4198,0.4133,0.4105,0.4144,0.4226,0.4123,0.4149,0.416,0.418,0.4226,0.4261


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0389,-0.0364,-0.0377,-0.0398,-0.0411,-0.0411,-0.0419,-0.0424,-0.0425,-0.0426,-0.0439,-0.0435,-0.0443,-0.0437,-0.0441
0.05,-0.0335,-0.0305,-0.0312,-0.0334,-0.0339,-0.0344,-0.035,-0.0356,-0.0358,-0.036,-0.0369,-0.0367,-0.0372,-0.037,-0.0369
0.25,-0.0196,-0.0164,-0.0163,-0.017,-0.0173,-0.0184,-0.0183,-0.0188,-0.0198,-0.0194,-0.0198,-0.0198,-0.0203,-0.0201,-0.0205
0.5,-0.0106,-0.0065,-0.0062,-0.0063,-0.0062,-0.0071,-0.0071,-0.0073,-0.0079,-0.0081,-0.0082,-0.0083,-0.0085,-0.0085,-0.0091
0.75,-0.0013,0.0036,0.0043,0.0048,0.0051,0.0046,0.0041,0.0042,0.0039,0.0035,0.0033,0.0033,0.003,0.0032,0.0029
0.95,0.0116,0.0179,0.0196,0.0208,0.0212,0.0205,0.0209,0.0207,0.0206,0.0199,0.0203,0.0198,0.0198,0.0204,0.0195
0.98,0.0176,0.0239,0.0256,0.0274,0.0276,0.0271,0.0276,0.0282,0.0276,0.0272,0.0273,0.0268,0.0268,0.0272,0.0267


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0037,-0.0035,-0.0029,-0.0029,-0.0027,-0.0023,-0.0023,-0.0023,-0.0018,-0.0019,-0.0019,-0.0013,-0.0013,-0.0012,-0.0008
0.05,-0.0012,-0.0003,0.0009,0.0012,0.0014,0.0017,0.0013,0.0019,0.0022,0.002,0.0021,0.0025,0.0024,0.0027,0.0028
0.25,0.0047,0.0077,0.0095,0.0104,0.0108,0.0111,0.0114,0.0116,0.0118,0.0119,0.0119,0.0119,0.0121,0.0122,0.0123
0.5,0.0086,0.0135,0.0156,0.0167,0.0174,0.0177,0.0182,0.0183,0.0183,0.0185,0.0185,0.0187,0.0189,0.0189,0.019
0.75,0.0126,0.0189,0.0217,0.0231,0.0238,0.0244,0.0248,0.0251,0.0251,0.0252,0.0251,0.0252,0.0255,0.0258,0.0259
0.95,0.0181,0.0271,0.0308,0.0324,0.0331,0.0336,0.0341,0.0345,0.035,0.035,0.035,0.0354,0.0355,0.0355,0.0352
0.98,0.0203,0.0305,0.0342,0.0364,0.0374,0.0377,0.038,0.0387,0.0387,0.0391,0.0392,0.0396,0.0396,0.0395,0.0391


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0024,-0.007,-0.0091,-0.01,-0.0102,-0.0102,-0.0098,-0.0106,-0.0108,-0.0104,-0.0105,-0.011,-0.0101,-0.0119,-0.0116
0.05,0.0033,-0.0011,-0.0028,-0.0037,-0.004,-0.0042,-0.0039,-0.0041,-0.0046,-0.0045,-0.0041,-0.0044,-0.004,-0.0059,-0.0047
0.25,0.0174,0.0128,0.0117,0.0111,0.0105,0.0106,0.0104,0.01,0.0103,0.0106,0.0102,0.0099,0.0102,0.0097,0.0102
0.5,0.0273,0.023,0.022,0.0214,0.0206,0.0208,0.0205,0.0203,0.0205,0.0204,0.0203,0.0202,0.0203,0.0203,0.02
0.75,0.037,0.033,0.032,0.0315,0.0312,0.0312,0.0308,0.0304,0.0304,0.0304,0.0306,0.0305,0.0304,0.0304,0.0303
0.95,0.0514,0.0471,0.0464,0.0464,0.0458,0.0457,0.0452,0.0451,0.0455,0.045,0.046,0.0454,0.0458,0.0448,0.0449
0.98,0.0572,0.0529,0.0525,0.0524,0.0523,0.0518,0.0512,0.0519,0.0516,0.0515,0.0521,0.0517,0.052,0.0511,0.0507


In [54]:
scenarios_b = simulate(bmodel,freq * 15, 10)

for n in 1:4
    print_percentiles(scenarios_b[n, :, 2:end], [0.05, 0.25, 0.5, 0.75, 0.95], freq, string(assets_names[n]))
 end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.2559,-0.7617,-0.8802,-0.9277,-0.9532,-0.9723,-0.978,-0.9854,-0.9934,-0.9994,-1.0092,-1.0087,-1.0158,-1.0192,-1.0243
0.25,0.0163,-0.2017,-0.2611,-0.2819,-0.2905,-0.2977,-0.3033,-0.3037,-0.3039,-0.3074,-0.3096,-0.3081,-0.3126,-0.3144,-0.3148
0.5,0.1755,0.1062,0.0887,0.0799,0.0813,0.0798,0.081,0.0809,0.0808,0.0805,0.0855,0.0828,0.0852,0.0847,0.0844
0.75,0.3531,0.4283,0.447,0.4498,0.4601,0.4644,0.4689,0.474,0.4724,0.4781,0.4795,0.4806,0.4855,0.4871,0.4889
0.95,0.679,1.0062,1.08,1.1157,1.1418,1.148,1.1601,1.1746,1.1857,1.1957,1.201,1.2105,1.2138,1.2315,1.236


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0604,-0.0964,-0.1127,-0.1231,-0.1307,-0.1368,-0.1424,-0.1467,-0.1504,-0.1529,-0.157,-0.159,-0.1617,-0.1647,-0.1668
0.25,-0.0287,-0.0391,-0.0451,-0.0488,-0.0516,-0.0542,-0.0558,-0.0571,-0.0583,-0.0595,-0.0604,-0.0609,-0.0615,-0.0615,-0.062
0.5,-0.0098,-0.0053,-0.005,-0.005,-0.0061,-0.0061,-0.0065,-0.0069,-0.0073,-0.0079,-0.0078,-0.0077,-0.0081,-0.0085,-0.0083
0.75,0.0085,0.0277,0.0347,0.038,0.0402,0.0412,0.0423,0.0428,0.043,0.0434,0.0436,0.0443,0.0441,0.0441,0.0444
0.95,0.0376,0.0833,0.1007,0.1103,0.1176,0.1228,0.1266,0.1299,0.1331,0.1361,0.1373,0.1378,0.1401,0.1425,0.1443


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0294,-0.0437,-0.0505,-0.0566,-0.0606,-0.0632,-0.0667,-0.0685,-0.0702,-0.073,-0.0746,-0.0752,-0.0763,-0.0776,-0.0792
0.25,-0.0064,-0.0087,-0.01,-0.011,-0.0117,-0.0122,-0.0124,-0.0128,-0.0129,-0.0131,-0.0134,-0.0136,-0.0135,-0.0138,-0.0141
0.5,0.0085,0.0131,0.015,0.0159,0.0166,0.0171,0.0175,0.0178,0.0181,0.0182,0.0185,0.0185,0.0189,0.0187,0.0188
0.75,0.0235,0.0346,0.0395,0.0426,0.0447,0.0462,0.0475,0.0482,0.0491,0.0497,0.0503,0.0507,0.0511,0.0514,0.0517
0.95,0.0461,0.0677,0.0782,0.0855,0.0917,0.0959,0.0996,0.1029,0.1064,0.1083,0.1101,0.1122,0.1135,0.1155,0.1176


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0012,-0.0174,-0.0239,-0.0274,-0.0304,-0.0324,-0.0343,-0.0363,-0.0376,-0.0394,-0.0407,-0.0417,-0.0424,-0.0433,-0.0442
0.25,0.0171,0.0084,0.0061,0.0046,0.0041,0.0033,0.0029,0.0027,0.0021,0.0019,0.0017,0.0015,0.0014,0.0012,0.0011
0.5,0.0277,0.0227,0.0218,0.0214,0.0212,0.021,0.0209,0.0209,0.0206,0.0206,0.0206,0.0206,0.0205,0.0205,0.0204
0.75,0.0381,0.0372,0.0377,0.0382,0.0385,0.0386,0.0385,0.0388,0.0389,0.0391,0.039,0.0392,0.0391,0.0392,0.0393
0.95,0.0549,0.0634,0.0676,0.0702,0.0726,0.0741,0.0749,0.0762,0.0775,0.0785,0.0788,0.0799,0.0801,0.0814,0.0821


In [55]:
# annualised compounded returns for 1, 5, 10, 15 years
for n in 1:3
    cum_ret = cumsum(scenarios[n,:,2:end]; dims=2)
    print_percentiles(expm1.(cum_ret[:,[1 * freq, freq * 5, freq * 10, freq * 15]] ./ [1 5 10 15]),[.02, .1, .25, .5, .75, .9, .98], 1, string(assets_names[n]))    
end

,1,2,3,4
0.02,-0.1107,-0.0178,0.0042,0.0169
0.1,0.0019,0.032,0.0404,0.0458
0.25,0.0979,0.0709,0.0684,0.0694
0.5,0.2089,0.1166,0.1008,0.0969
0.75,0.33,0.1639,0.1337,0.1238
0.9,0.4502,0.2084,0.1637,0.1479
0.98,0.6185,0.2678,0.2056,0.1832


,1,2,3,4
0.02,-0.0382,-0.0226,-0.0219,-0.0214
0.1,-0.0279,-0.0169,-0.0165,-0.0163
0.25,-0.0194,-0.0121,-0.0121,-0.0122
0.5,-0.0106,-0.0071,-0.0073,-0.0077
0.75,-0.0013,-0.0019,-0.0024,-0.0031
0.9,0.0069,0.003,0.002,0.0011
0.98,0.0178,0.0087,0.0077,0.0065


,1,2,3,4
0.02,-0.0037,0.0012,0.0041,0.006
0.1,0.001,0.0062,0.0087,0.0102
0.25,0.0047,0.01,0.0123,0.0135
0.5,0.0087,0.0145,0.0164,0.0172
0.75,0.0127,0.0188,0.0205,0.021
0.9,0.0162,0.0228,0.0242,0.0244
0.98,0.0206,0.0278,0.0287,0.0286


In [56]:
collect(.05:.1:.95)

10-element Vector{Float64}:
 0.05
 0.15
 0.25
 0.35
 0.45
 0.55
 0.65
 0.75
 0.85
 0.95

In [57]:
for n in 1:3
    print_percentiles(scenarios[n, :, 2:end],[.05, .15, .25, .35, .45, .55, .65, 0.75, .85, 0.95], freq*5, string(assets_names[n]))
end


,1,2,3
0.05,0.048,-0.1936,-0.1692
0.15,0.2325,0.0354,0.052
0.25,0.3426,0.1655,0.1763
0.35,0.431,0.2697,0.2835
0.45,0.5121,0.3659,0.3762
0.55,0.5884,0.4557,0.4652
0.65,0.6719,0.5464,0.5622
0.75,0.759,0.6468,0.6704
0.85,0.8721,0.7799,0.8003
0.95,1.0592,0.988,1.0207


,1,2,3
0.05,-0.0986,-0.1231,-0.1306
0.15,-0.0751,-0.09,-0.097
0.25,-0.061,-0.0718,-0.0778
0.35,-0.0503,-0.0575,-0.0619
0.45,-0.0405,-0.0445,-0.0486
0.55,-0.0306,-0.0309,-0.0361
0.65,-0.0212,-0.0175,-0.0226
0.75,-0.0093,-0.0027,-0.0077
0.85,0.0047,0.0165,0.0117
0.95,0.0283,0.0488,0.0449


,1,2,3
0.05,0.019,0.026,0.0287
0.15,0.0384,0.0507,0.0528
0.25,0.0497,0.0645,0.0676
0.35,0.0591,0.0758,0.079
0.45,0.068,0.0861,0.0891
0.55,0.0755,0.096,0.099
0.65,0.084,0.1059,0.109
0.75,0.0933,0.1171,0.1206
0.85,0.1045,0.1319,0.1356
0.95,0.1248,0.1554,0.1598


In [58]:
 cov_m = cov(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI])) 
 mu = mean(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI]), dims=1) 

KeyError: KeyError: key :Price_10Y_T_Bill not found

In [59]:
using Distributions
scenarios_n = zeros(3,100_000,freq*15)

for t in 1:freq*15
    scenarios_n[:,:,t] = rand(MvNormal(vec(mu), cov_m), 100_000)
end



UndefVarError: UndefVarError: `mu` not defined

In [60]:
for n in 1:3
    print_percentiles(scenarios_n[n, :, 1:end],[0.02, 0.25, 0.5, 0.75, 0.98], 12, string(assets_names[n]))
end

,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


In [61]:
returns_summarystats(data_source[:Yeld_10Y],1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Yeld_10Y,0.0292,0.1584,0.0548,0.0736,0.0066,0.9377,0.0333,0.9774,0.6945,1.8772,0.0468


In [62]:
yelds10y = dropdims(sum(scenarios[[3,5,6],:,:], dims=1), dims=1)
print_percentiles(annualise(yelds10y,4), [0.02, 0.25, 0.5, 0.75, 0.98])

BoundsError: BoundsError: attempt to access 4×10000×61 Array{Float64, 3} at index [[3, 5, 6], 1:10000, 1:61]

In [63]:
include("generate_bond_temp.jl")

using .Bonds

bond_returns = Bonds.scenarios_bond_returns(yelds10y, 10, freq)
print_percentiles(bond_returns, [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `yelds10y` not defined

In [64]:
#cpi_ann = annualise(scenarios[6,:,2:end],4)
t_bill = scenarios[3,:,2:end]
excess_bond_returns = bond_returns .- t_bill
print_percentiles(expm1.(excess_bond_returns), [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `bond_returns` not defined

In [65]:
scenario = 3
plot(yelds10y[scenario,:], label="yeld")
plot!(twinx(), bond_returns[scenario,:], label="bond returns", color=:orange)

UndefVarError: UndefVarError: `yelds10y` not defined

In [66]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

5-element Vector{Float64}:
 -0.18481487096322075
 -0.02926484511596377
  0.07929430075546959
  0.1876537157297006
  0.34319841876835994

In [67]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

5-element Vector{Float64}:
 -0.1827572813665558
 -0.02855002826502162
  0.08028463118450836
  0.18797849002436737
  0.3445804547664945

In [68]:
simt = rand(LocationScale(.04, .075, TDist(2.5)), 100_000)
quantile(simt, [.02, .25, .5, .75, .98])

5-element Vector{Float64}:
 -0.2620107536011347
 -0.018785104869753697
  0.04009567121135338
  0.09902260095709473
  0.34229136889914147